In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
import zipfile
import os


zip_file_path = '/content/drive/MyDrive/paper1/Level2.zip' 


extracted_folder_path = '/content/drive/MyDrive/paper1/'  


os.makedirs(extracted_folder_path, exist_ok=True)


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)


extracted_contents = os.listdir(extracted_folder_path)
print(f"Contents of the extraction folder: {extracted_contents}")


Contents of the extraction folder: ['final_folder.zip', 'Level2.zip', 'level3_diffusion.zip', 'final_folder', '__MACOSX', 'Level2']


In [10]:
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/paper1/Level2', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

In [11]:
resnet18 = models.resnet18(pretrained=True)


num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 2)


resnet18 = resnet18.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.005, momentum=0.8)

In [12]:
num_epochs = 4
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)


        optimizer.zero_grad()


        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)


        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

print("Training finished")

Epoch 1/4, Loss: 0.412191071386995
Epoch 2/4, Loss: 0.26303275391973296
Epoch 3/4, Loss: 0.19956664689655962
Epoch 4/4, Loss: 0.15324576789962835
Training finished


In [13]:
torch.save(resnet18.state_dict(), '/content/drive/MyDrive/paper1/fine_tuned_resnet18_level2.pth')

In [14]:
import torch
from torchvision import transforms
from PIL import Image


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


model = models.resnet18()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) 
model.load_state_dict(torch.load('/content/drive/MyDrive/paper1/fine_tuned_resnet18_level2.pth'))
model.eval()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [16]:

image_path = '/content/drive/MyDrive/paper1/Level2/gan_final/00000737.png'
image = Image.open(image_path).convert('RGB')
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0).to(device)  


with torch.no_grad():
    output = model(input_batch)


probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()


print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")

Predicted class: 1
Class probabilities: tensor([0.1914, 0.8086])


In [21]:
import os
from PIL import Image


folder_path = '/content/drive/MyDrive/paper1/level3_diffusion/'


l = ["glide_100_10","idm_100_cfg"]

for i in l:
  files = os.listdir(folder_path+i)


  image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
  count = 1
  count2 = 0

  for image_file in image_files[:100]:

    image_path = os.path.join(folder_path+i, image_file)

    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image)
        input_batch = input_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
          output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()


        print(f"Predicted class: {predicted_class}")
        print(f"Class probabilities: {probabilities}")

        if(predicted_class == 0):
          print("yes")
          count2 += 1
        image.close()
        print(count)
        count += 1
        print(f"Opened image: {image_path}")
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
  print(f"the percentage of predicting {i} images is : {(count2*100)/count}")

Predicted class: 0
Class probabilities: tensor([0.9022, 0.0978])
yes
1
Opened image: /content/drive/MyDrive/paper1/level3_diffusion/glide_100_10/vmlzuywhlr.png
Predicted class: 0
Class probabilities: tensor([0.9973, 0.0027])
yes
2
Opened image: /content/drive/MyDrive/paper1/level3_diffusion/glide_100_10/sdpkvjboba.png
Predicted class: 1
Class probabilities: tensor([0.4724, 0.5276])
3
Opened image: /content/drive/MyDrive/paper1/level3_diffusion/glide_100_10/nnuuhindgj.png
Predicted class: 0
Class probabilities: tensor([0.9967, 0.0033])
yes
4
Opened image: /content/drive/MyDrive/paper1/level3_diffusion/glide_100_10/uxbuickkha.png
Predicted class: 0
Class probabilities: tensor([0.9982, 0.0018])
yes
5
Opened image: /content/drive/MyDrive/paper1/level3_diffusion/glide_100_10/echehlitmr.png
Predicted class: 0
Class probabilities: tensor([0.9859, 0.0141])
yes
6
Opened image: /content/drive/MyDrive/paper1/level3_diffusion/glide_100_10/zxaydtvcbx.png
Predicted class: 0
Class probabilities: tens

In [24]:
import os
from PIL import Image


folder_path = '/content/drive/MyDrive/paper1/level3_gans_1/'


l = ["gaugan","stylegan"]

for i in l:
  files = os.listdir(folder_path+i)


  image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
  count = 1
  count2 = 0

  for image_file in image_files[:100]:

    image_path = os.path.join(folder_path+i, image_file)

    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image)
        input_batch = input_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
          output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()


        print(f"Predicted class: {predicted_class}")
        print(f"Class probabilities: {probabilities}")

        if(predicted_class == 1):
          print("yes")
          count2 += 1
        image.close()
        print(count)
        count += 1
        print(f"Opened image: {image_path}")
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
  print(f"the percentage of predicting {i} images is : {(count2*100)/count}")

Predicted class: 0
Class probabilities: tensor([0.9314, 0.0686])
1
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/gaugan/000000117645.png
Predicted class: 0
Class probabilities: tensor([0.9280, 0.0720])
2
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/gaugan/000000104612.png
Predicted class: 1
Class probabilities: tensor([0.2515, 0.7485])
yes
3
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/gaugan/000000082846.png
Predicted class: 0
Class probabilities: tensor([0.8169, 0.1831])
4
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/gaugan/000000074646.png
Predicted class: 0
Class probabilities: tensor([0.9947, 0.0053])
5
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/gaugan/000000125806.png
Predicted class: 1
Class probabilities: tensor([0.1478, 0.8522])
yes
6
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/gaugan/000000119641.png
Predicted class: 1
Class probabilities: tensor([0.1457, 0.8543])
yes
7
Opened image: /content/driv

In [25]:
import os
from PIL import Image


folder_path = '/content/drive/MyDrive/paper1/level3_gans_1/'


l = ["biggan","stargan"]

for i in l:
  files = os.listdir(folder_path+i)

  image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
  count = 1
  count2 = 0

  for image_file in image_files[:100]:

    image_path = os.path.join(folder_path+i, image_file)


    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image)
        input_batch = input_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
          output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()


        print(f"Predicted class: {predicted_class}")
        print(f"Class probabilities: {probabilities}")

        if(predicted_class == 1):
          print("yes")
          count2 += 1
        image.close()
        print(count)
        count += 1
        print(f"Opened image: {image_path}")
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
  print(f"the percentage of predicting {i} images is : {(count2*100)/count}")

Predicted class: 1
Class probabilities: tensor([0.0536, 0.9464])
yes
1
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/biggan/00290310.png
Predicted class: 1
Class probabilities: tensor([0.2366, 0.7634])
yes
2
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/biggan/00305326.png
Predicted class: 1
Class probabilities: tensor([0.0614, 0.9386])
yes
3
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/biggan/00292728.png
Predicted class: 0
Class probabilities: tensor([0.8297, 0.1703])
4
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/biggan/00146270.png
Predicted class: 1
Class probabilities: tensor([0.2829, 0.7171])
yes
5
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/biggan/00158538.png
Predicted class: 1
Class probabilities: tensor([0.4573, 0.5427])
yes
6
Opened image: /content/drive/MyDrive/paper1/level3_gans_1/biggan/00145596.png
Predicted class: 1
Class probabilities: tensor([0.2197, 0.7803])
yes
7
Opened image: /content/drive/MyDrive/pa

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
